In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
# import os

# os.makedirs('/kaggle/working/valid_dataset/finger/', exist_ok=True)
# os.makedirs('/kaggle/working/valid_dataset/hand/', exist_ok=True)
# os.makedirs('/kaggle/working/valid_dataset/forearm/', exist_ok=True)
# os.makedirs('/kaggle/working/valid_dataset/humerus/', exist_ok=True)
# os.makedirs('/kaggle/working/valid_dataset/elbow/', exist_ok=True)

In [3]:
# import os
# import shutil
# import random

# source_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/elbow'
# dest_dir = '/kaggle/working/valid_dataset/elbow/'

# files = os.listdir(source_dir)
# random.shuffle(files)
# num_val_files = int(len(files)*0.10)
# random_val_files  = random.sample(files, num_val_files)

# for file in random_val_files:
#     file_path = os.path.join(source_dir, file)
#     dest_path = os.path.join(dest_dir, file)
#     shutil.move(file_path, dest_path)

In [4]:
# import os
# import shutil
# import random

# source_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/finger'
# dest_dir = '/kaggle/working/valid_dataset/finger/'

# files = os.listdir(source_dir)
# random.shuffle(files)
# num_val_files = int(len(files)*0.10)
# random_val_files  = random.sample(files, num_val_files)

# for file in random_val_files:
#     file_path = os.path.join(source_dir, file)
#     dest_path = os.path.join(dest_dir, file)
#     shutil.move(file_path, dest_path)
    

In [5]:
# import os
# import shutil
# import random

# source_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/hand'
# dest_dir = '/kaggle/working/valid_dataset/hand/'

# files = os.listdir(source_dir)
# random.shuffle(files)
# num_val_files = int(len(files)*0.10)
# random_val_files  = random.sample(files, num_val_files)

# for file in random_val_files:
#     file_path = os.path.join(source_dir, file)
#     dest_path = os.path.join(dest_dir, file)
#     shutil.move(file_path, dest_path)
    

In [6]:
# import os
# import shutil
# import random

# source_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/humerus'
# dest_dir = '/kaggle/working/valid_dataset/humerus/'

# files = os.listdir(source_dir)
# random.shuffle(files)
# num_val_files = int(len(files)*0.10)
# random_val_files  = random.sample(files, num_val_files)

# for file in random_val_files:
#     file_path = os.path.join(source_dir, file)
#     dest_path = os.path.join(dest_dir, file)
#     shutil.move(file_path, dest_path)
    

In [7]:
# import os
# import shutil
# import random

# source_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/forearm'
# dest_dir = '/kaggle/working/valid_dataset/forearm/'

# files = os.listdir(source_dir)
# random.shuffle(files)
# num_val_files = int(len(files)*0.10)
# random_val_files  = random.sample(files, num_val_files)

# for file in random_val_files:
#     file_path = os.path.join(source_dir, file)
#     dest_path = os.path.join(dest_dir, file)
#     shutil.move(file_path, dest_path)
    

In [8]:
# re-size all the images to this
IMAGE_SIZE = [350, 350]

train_path = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/'
valid_path = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/'

In [9]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 0s 0us/step


In [10]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [11]:
folders = glob('/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/*')

In [12]:
x = Flatten()(vgg.output)


In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

prediction

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

In [14]:
model = Model(inputs=vgg.input, outputs=prediction)


In [15]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 350, 350, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 350, 350, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 350, 350, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 175, 175, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 175, 175, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 175, 175, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 87, 87, 128)       0     

In [16]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# valid_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
base_dir = '/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset'

training_set = train_datagen.flow_from_directory(base_dir,
                                                         target_size=(350, 350),
                                                         batch_size=32,
                                                         class_mode='categorical')

# valid_set = valid_datagen.flow_from_directory('/kaggle/working/valid_dataset/',
#                                                  target_size = (350, 350),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')

Found 2188 images belonging to 5 classes.


In [19]:
import glob


r = model.fit_generator(
  training_set,
#   validation_data=valid_set,
  epochs=10,
  steps_per_epoch=len(training_set),
#   validation_steps=len(valid_set)
)
   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
69/69 [==============================] - 95s 1s/step - loss: 1.4442 - accuracy: 0.6060
Epoch 2/10
69/69 [==============================] - 70s 1s/step - loss: 0.5051 - accuracy: 0.8099
Epoch 3/10
69/69 [==============================] - 70s 1s/step - loss: 0.3727 - accuracy: 0.8633
Epoch 4/10
69/69 [==============================] - 70s 1s/step - loss: 0.2645 - accuracy: 0.9045
Epoch 5/10
69/69 [==============================] - 71s 1s/step - loss: 0.2686 - accuracy: 0.9027
Epoch 6/10
69/69 [==============================] - 69s 1s/step - loss: 0.2639 - accuracy: 0.9049
Epoch 7/10
69/69 [==============================] - 70s 1s/step - loss: 0.1760 - accuracy: 0.9429
Epoch 8/10
69/69 [==============================] - 71s 1s/step - loss: 0.1556 - accuracy: 0.9397
Epoch 9/10
69/69 [==============================] - 69s 999ms/step - loss: 0.1356 - accuracy: 0.9561
Epoch 10/10
69/69 [==============================] - 70s 1s/step - loss: 0.1200 - accuracy: 0.9612


In [20]:
import os

print(os.path.exists('/kaggle/input/multi-class-dataset/mutli_class_augmented/train_dataset/'))


True


In [21]:
# # plot the loss
# plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
# plt.legend()
# plt.show()
# plt.savefig('LossVal_loss')

# # plot the accuracy
# plt.plot(r.history['accuracy'], label='train acc')
# plt.plot(r.history['val_accuracy'], label='val acc')
# plt.legend()
# plt.show()
# plt.savefig('AccVal_acc')

In [22]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16_multiclass_2.h5')

# Test

In [23]:
from keras.models import load_model

model = load_model('/kaggle/working/model_vgg16_multiclass_2.h5')

In [24]:
from keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
test_set = test_datagen.flow_from_directory('/kaggle/input/multi-class-dataset/mutli_class_augmented/test_dataset',
                                             target_size=(350, 350),
                                             batch_size=32,
                                             class_mode='categorical')

Found 383 images belonging to 5 classes.


In [26]:
loss, accuracy = model.evaluate(test_set)

print('Test set accuracy:', accuracy)

12/12 [==============================] - 16s 1s/step - loss: 0.2905 - accuracy: 0.8930
Test set accuracy: 0.8929504156112671


In [27]:
# plt.figure(figsize=(8, 4))
# plt.subplot(1, 2, 1)
# plt.plot(loss, label='Test Loss')
# plt.title('Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(accuracy, label='Test Accuracy')
# plt.title('Test Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()